In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import ast
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

In [71]:
df = pd.read_csv('prediction.csv')
df.head()

,cdr3,crd3_encode,j.segm,v.seg,mhc.a,mhc.b,v.seg_0,v.seg_1,v.seg_2,v.seg_3,...,mhc.a_1,mhc.a_2,mhc.a_3,mhc.a_4,mhc.a_5,mhc.a_6,mhc.b_label,mhc.class,antigen.epitope,vdjdb.score
0,CAAAERNTGELFF,"[-0.16498591005802155, -0.1462753862142563, 0....",TRBJ2-2*01,TRBV28*01,HLA-A*02,B2M,0.0,1.0,1.0,0.0,...,0,0,0,1,0,0,0,0,YLQPRTFLL,0
1,CAAEDPEWGAEAFF,"[-0.24984240531921387, 0.38962453603744507, -0...",TRBJ1-1*01,TRBV30*01,HLA-A*03:01,B2M,0.0,1.0,1.0,0.0,...,0,0,1,1,1,1,0,0,KLGGALQAK,0
2,CAAGAGLSYEQYF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-7*01,TRBV27*01,HLA-A*02,B2M,0.0,1.0,0.0,1.0,...,0,0,0,1,0,0,0,0,NLVPMVATV,0
3,CAAGDANTGELFF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-2*01,TRBV5-1*01,HLA-A*02:01,B2M,0.0,1.0,1.0,1.0,...,0,0,0,1,0,1,0,0,YLQPRTFLL,0
4,CAAGEMFGLGETQYF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-5*01,TRBV4-2*01,HLA-A*03:01,B2M,0.0,1.0,1.0,1.0,...,0,0,1,1,1,1,0,0,KLGGALQAK,0


In [72]:
encoder = LabelEncoder()
label_rencoded = encoder.fit_transform(df['antigen.epitope'].tolist())
df['label_rencoded'] = label_rencoded
df.head()

,cdr3,crd3_encode,j.segm,v.seg,mhc.a,mhc.b,v.seg_0,v.seg_1,v.seg_2,v.seg_3,...,mhc.a_2,mhc.a_3,mhc.a_4,mhc.a_5,mhc.a_6,mhc.b_label,mhc.class,antigen.epitope,vdjdb.score,label_rencoded
0,CAAAERNTGELFF,"[-0.16498591005802155, -0.1462753862142563, 0....",TRBJ2-2*01,TRBV28*01,HLA-A*02,B2M,0.0,1.0,1.0,0.0,...,0,0,1,0,0,0,0,YLQPRTFLL,0,12
1,CAAEDPEWGAEAFF,"[-0.24984240531921387, 0.38962453603744507, -0...",TRBJ1-1*01,TRBV30*01,HLA-A*03:01,B2M,0.0,1.0,1.0,0.0,...,0,1,1,1,1,0,0,KLGGALQAK,0,6
2,CAAGAGLSYEQYF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-7*01,TRBV27*01,HLA-A*02,B2M,0.0,1.0,0.0,1.0,...,0,0,1,0,0,0,0,NLVPMVATV,0,8
3,CAAGDANTGELFF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-2*01,TRBV5-1*01,HLA-A*02:01,B2M,0.0,1.0,1.0,1.0,...,0,0,1,0,1,0,0,YLQPRTFLL,0,12
4,CAAGEMFGLGETQYF,"[-0.7613160014152527, 0.263674259185791, -0.52...",TRBJ2-5*01,TRBV4-2*01,HLA-A*03:01,B2M,0.0,1.0,1.0,1.0,...,0,1,1,1,1,0,0,KLGGALQAK,0,6


In [73]:
X = df[['crd3_encode', 'v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5','vdjdb.score',
              'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3',
              'mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label', 'mhc.class']].fillna(0)
y = df['label_rencoded']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [75]:
X_train_copy_list = []
y_train_copy_list = []

for index, row in X_train.iterrows():
    num_copies = row['vdjdb.score']
    if num_copies > 0:
        X_train_copy_list.extend([row] * num_copies)
        y_train_copy_list.extend([y_train[index]] * num_copies)
X_train_copy = pd.concat(X_train_copy_list, axis=1).T
y_train_copy = pd.Series(y_train_copy_list, name='label_rencoded')

X_train_copy.reset_index(drop=True, inplace=True)
y_train_copy.reset_index(drop=True, inplace=True)

X_train_new = pd.concat([X_train, X_train_copy], ignore_index=True)
y_train_new = pd.concat([y_train, y_train_copy], ignore_index=True)

X_train_new.reset_index(drop=True, inplace=True)
y_train_new.reset_index(drop=True, inplace=True)

In [76]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn import manifold

def dimensionality_reduction(df, n_components1, n_components2):
    vdj = df[['v.seg_0', 'v.seg_1', 'v.seg_2', 'v.seg_3', 'v.seg_4', 'v.seg_5',
              'j.segm_0', 'j.segm_1', 'j.segm_2', 'j.segm_3']]
    mhc = df[['mhc.a_0', 'mhc.a_1', 'mhc.a_2', 'mhc.a_3', 'mhc.a_4', 'mhc.a_5', 'mhc.a_6', 'mhc.b_label']]
    cdr3_kmer = np.array(df['crd3_encode'].apply(ast.literal_eval).tolist())
    pca = PCA(n_components=n_components1)
    pca_data = pca.fit_transform(cdr3_kmer)

    tcr_list = np.array([list(cdr3) + vdj_row.tolist() + mhc_row.tolist() for cdr3, vdj_row, mhc_row in zip(pca_data, vdj.values, mhc.values)])

    tsne = manifold.TSNE(n_components=n_components2, init='pca', random_state=1, learning_rate=100)
    reduced_data = tsne.fit_transform(tcr_list)

    return pca_data, reduced_data


In [77]:
train_pca, X_train_preprocessing = dimensionality_reduction(X_train_new, 50, 3)
test_pca, X_test_preprocessing = dimensionality_reduction(X_test, 50, 3)

In [78]:
train_pca

array([[-10.96126057,   1.97835571,  -0.51224997, ...,  -2.71837375,
          0.69450782,  -0.32707129],
       [ -6.2622007 ,   9.53711246,  -4.25910831, ...,  -0.13567384,
         -1.07015328,   0.26373344],
       [  9.04788519,   0.07170993,  -4.18383011, ...,   0.59984318,
          0.9816438 ,  -0.13462531],
       ...,
       [ -0.41084046,  -2.28976361,   5.73912047, ...,  -0.38085488,
          0.62433255,  -1.87300779],
       [ -4.57218909,  -5.15388255,   1.23842235, ...,   1.92078854,
          3.95398651,  -0.46931333],
       [ -4.68962663,  -6.47275764,  -3.29646548, ...,  -0.33116489,
          0.73163187,   1.02383857]])

In [79]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_new), y=y_train_new)
class_weights

array([1.49235369, 1.4521958 , 4.13409779, 0.61439053, 1.81855121,
       4.23044329, 0.19589613, 3.94618425, 0.43923652, 2.13330046,
       4.37887989, 4.61147628, 2.28987297])

In [80]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [82]:
def custom_scorer(y_true, y_pred):
    auc_scores = []
    for i in range (len(np.unique(y_true))):
        y_pred_class = y_pred[:, i]
        y_true_class = (y_true == i)
        auc_scores.append(roc_auc_score(y_true_class, y_pred_class))
    auc_score = sum(auc_scores) / len(np.unique(y_train))
    return auc_score

custom_scorer = make_scorer(custom_scorer)

## SVM

In [83]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_preprocessed = scaler.fit_transform(X_train_preprocessing)
X_test_preprocessed = scaler.transform(X_test_preprocessing)


In [84]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

svm_model = OneVsRestClassifier(SVC(kernel='linear', probability=True))
svm_model.fit(X_train_preprocessed, y_train_new)
svm_pred = svm_model.predict_proba(X_test_preprocessed)


In [85]:
auc_scores_svm = []
for i in range(len(np.unique(y_test))):
    y_pred_class_svm = svm_pred[:, i]
    y_true_class = (y_test == i)
    auc_scores_svm.append(roc_auc_score(y_true_class, y_pred_class_svm))
mean_auc_svm = sum(auc_scores_svm) / len(auc_scores_svm)

print('SVM AUC: ', mean_auc_svm)


SVM AUC:  0.55579715119162


In [100]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

svm_poly_model = OneVsRestClassifier(SVC(kernel='poly', degree=5, C=1, gamma='scale', probability=True))
svm_poly_model.fit(X_train_preprocessed, y_train_new)
svm_poly_pred = svm_poly_model.predict_proba(X_test_preprocessed)

auc_scores_poly = []
for i in range(len(np.unique(y_test))):
    y_pred_class_poly = svm_poly_pred[:, i]
    y_true_class = (y_test == i)
    auc_scores_poly.append(roc_auc_score(y_true_class, y_pred_class_poly))
mean_auc_poly = sum(auc_scores_poly) / len(auc_scores_poly)

print('Poly SVM AUC: ', mean_auc_poly)


Poly SVM AUC:  0.5743057604514127


## LogisticRegression

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_preprocessed = scaler.fit_transform(X_train_preprocessing)
X_test_preprocessed = scaler.transform(X_test_preprocessing)

logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
logistic_model.fit(X_train_preprocessed, y_train_new)
logistic_pred = logistic_model.predict_proba(X_test_preprocessed)
auc_scores_logistic = []
for i in range(len(np.unique(y_test))):
    y_pred_class_logistic = logistic_pred[:, i]
    y_true_class = (y_test == i)
    auc_scores_logistic.append(roc_auc_score(y_true_class, y_pred_class_logistic))
mean_auc_logistic = sum(auc_scores_logistic) / len(auc_scores_logistic)

print('Logistic Regression AUC: ', mean_auc_logistic)


Logistic Regression AUC:  0.5955290060528479


In [93]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter': [100, 200, 500, 1000]
}
logistic_model = LogisticRegression(multi_class='multinomial')

grid_search = GridSearchCV(estimator=logistic_model, param_grid=param_grid, cv=5, verbose=2, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_preprocessed, y_train_new)
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validated accuracy: {:.2f}%".format(grid_search.best_score_ * 100))


Fitting 5 folds for each of 120 candidates, totalling 600 fits


/Users/julio/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
120 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/julio/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/julio/anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/julio/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py", line 1212, in fit
    multi_class = _check_

Best parameters found:  {'C': 1, 'max_iter': 100, 'solver': 'newton-cg'}
Best cross-validated accuracy: 39.87%


In [94]:
optimized_logistic = LogisticRegression(C=1, max_iter=100, solver='newton-cg', multi_class='multinomial')
optimized_logistic.fit(X_train_preprocessed, y_train_new)

optimized_pred = optimized_logistic.predict_proba(X_test_preprocessed)

optimized_auc_scores = []
for i in range(len(np.unique(y_test))):
    optimized_pred_class = optimized_pred[:, i]
    optimized_true_class = (y_test == i)
    optimized_auc_scores.append(roc_auc_score(optimized_true_class, optimized_pred_class))

optimized_mean_auc = sum(optimized_auc_scores) / len(optimized_auc_scores)
print('Optimized Logistic Regression AUC: ', optimized_mean_auc)


Optimized Logistic Regression AUC:  0.5954840026537439
